In [12]:
import torch 
import torch.nn as nn
import torch.nn.functional as F 
import torch.utils.data as Data 
import torch.optim as optim
import torchvision

In [13]:
batch_size = 64
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    # 切换到标准差分布
    torchvision.transforms.Normalize((0.1307, ), (0.3081, ))
])

In [14]:
train_data = torchvision.datasets.MNIST('./dataset/mnist/', train= True, transform= transform)
train_loader = Data.DataLoader(train_data, shuffle= True, batch_size= batch_size)
test_date = torchvision.datasets.MNIST('./dataset/mnist/', train= False, transform= transform)
test_loader = Data.DataLoader(test_date, shuffle= False, batch_size= batch_size)


In [15]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 512)
        self.l2 = nn.Linear(512, 256)
        self.l3 = nn.Linear(256, 128)
        self.l4 = nn.Linear(128, 64)
        self.l5 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        # 注意最后一层不做激活 因为输出要接到后面的softmax里面
        return self.l5(x)

model = Net()

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum= 0.5)

In [17]:
def train(epoch):
    running_loss = 0
    for batch_idx, data in enumerate(train_loader):
        inputs, target = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 300 ==299:
            print("[%d,%5d] loss: %.3f" % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0


    

In [18]:
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            outputs = model(inputs)
            _, prediction = torch.max(outputs.data, dim = 1)
            total += labels.size(0)
            correct += (prediction == labels).sum().item()
    print("ACC: %d %%" % (100*correct/total))

In [19]:
if __name__ == '__main__':
    for epoch in range(5):
        train(epoch)
        test()

[1,  300] loss: 2.182
[1,  600] loss: 0.837
[1,  900] loss: 0.436
ACC: 90 %
[2,  300] loss: 0.324
[2,  600] loss: 0.267
[2,  900] loss: 0.238
ACC: 94 %
[3,  300] loss: 0.196
[3,  600] loss: 0.164
[3,  900] loss: 0.163
ACC: 95 %
[4,  300] loss: 0.135
[4,  600] loss: 0.121
[4,  900] loss: 0.121
ACC: 96 %
[5,  300] loss: 0.105
[5,  600] loss: 0.092
[5,  900] loss: 0.092
ACC: 96 %
